#### Importing necessary libraries

In [1]:
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

#### Ignoring warnings and setting proper display options

In [6]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

#### Loading required fantasy data sets

In [9]:
df1 = pd.read_csv('ind_sl_t20i_1st_2022_02_24.csv')
df2 = pd.read_csv('ind_sl_t20i_2nd_2022_02_26.csv')
df3 = pd.read_csv('ind_sl_t20i_3rd_2022_02_27.csv')

#### Filtering to extract required columns

In [11]:
df1 = df1[['Player', 'Team', 'Role', 'Credits', 'Points', 'PlayingXI']]
df2 = df2[['Player', 'Team', 'Role', 'Credits', 'Points', 'PlayingXI']]
df3 = df3[['Player', 'Team', 'Role', 'Credits', 'Points', 'PlayingXI']]

#### IND VS SL 1ST T20I

In [13]:
# Filtering df1 to select the players announced in Playing XI
df1 = df1[df1['PlayingXI'] == 1]

In [15]:
# checking the data in df1
df1

,Player,Team,Role,Credits,Points,PlayingXI
0,I Kishan,IND,WK,8.5,141,1
1,S Iyer,IND,BAT,8.5,84,1
2,B Kumar,IND,BOWL,8.5,76,1
3,C Asalanka,SL,BAT,8.5,70,1
4,R Sharma,IND,BAT,10.5,66,1
5,V Iyer,IND,AR,8.5,50,1
6,Y Chahal,IND,BOWL,8.5,35,1
7,L Kumara,SL,BOWL,8.5,35,1
8,R Jadeja,IND,AR,9.0,34,1
9,D Shanaka,SL,AR,9.0,32,1


In [17]:
# converting team names and roles into a numerical format that machine learning algorithms can work with effectively 
# and joininig them with df1
df1 = df1.join(pd.get_dummies(df1[['Role', 'Team']]))

In [19]:
df1

,Player,Team,Role,Credits,Points,PlayingXI,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,I Kishan,IND,WK,8.5,141,1,False,False,False,True,True,False
1,S Iyer,IND,BAT,8.5,84,1,False,True,False,False,True,False
2,B Kumar,IND,BOWL,8.5,76,1,False,False,True,False,True,False
3,C Asalanka,SL,BAT,8.5,70,1,False,True,False,False,False,True
4,R Sharma,IND,BAT,10.5,66,1,False,True,False,False,True,False
5,V Iyer,IND,AR,8.5,50,1,True,False,False,False,True,False
6,Y Chahal,IND,BOWL,8.5,35,1,False,False,True,False,True,False
7,L Kumara,SL,BOWL,8.5,35,1,False,False,True,False,False,True
8,R Jadeja,IND,AR,9.0,34,1,True,False,False,False,True,False
9,D Shanaka,SL,AR,9.0,32,1,True,False,False,False,False,True


In [21]:
# define decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status', ((obj) for obj in df1.index), cat = 'Binary')

# define objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Points'] for obj in df1.index])

# define constraints
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Credits']] for obj in df1.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'PlayingXI']] for obj in df1.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Team_IND']] for obj in df1.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Team_IND']] for obj in df1.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Team_SL']] for obj in df1.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Team_SL']] for obj in df1.index) <= 7

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_WK']] for obj in df1.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_WK']] for obj in df1.index) <= 3

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BAT']] for obj in df1.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BAT']] for obj in df1.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BOWL']] for obj in df1.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BOWL']] for obj in df1.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_AR']] for obj in df1.index) >= 2
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_AR']] for obj in df1.index) <= 4



In [23]:
# solving the linear programming problem for the given constraints
mo.solve()

1

In [25]:
# output as a column in df
df1['pickup_status'] = 0

for obj in df1.index:
    if pickup_status[obj].varValue:
        df1['pickup_status'][obj] = 1

In [27]:
# Keeping only the players picked up in the fantasy team
fdf1 = df1[df1['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)

#### 1ST T20I Fantasy Team

In [30]:
fdf1

,Player,Team,Role,Credits,Points,PlayingXI,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,S Iyer,IND,BAT,8.5,84,1,False,True,False,False,True,False,1
1,C Asalanka,SL,BAT,8.5,70,1,False,True,False,False,False,True,1
2,R Sharma,IND,BAT,10.5,66,1,False,True,False,False,True,False,1
3,I Kishan,IND,WK,8.5,141,1,False,False,False,True,True,False,1
4,V Iyer,IND,AR,8.5,50,1,True,False,False,False,True,False,1
5,R Jadeja,IND,AR,9.0,34,1,True,False,False,False,True,False,1
6,D Shanaka,SL,AR,9.0,32,1,True,False,False,False,False,True,1
7,B Kumar,IND,BOWL,8.5,76,1,False,False,True,False,True,False,1
8,Y Chahal,IND,BOWL,8.5,35,1,False,False,True,False,True,False,1
9,L Kumara,SL,BOWL,8.5,35,1,False,False,True,False,False,True,1


In [32]:
sum(fdf1.Credits)

97.0

In [34]:
# Total Points in the first fantasy team
F1TPS = sum(fdf1['Points'])

F1TPS

653

#### IND VS SL 2ND T20I

In [36]:
# Filtering df2 to select the players announced in Playing XI
df2 = df2[df2['PlayingXI'] == 1]

In [38]:
# converting team names and roles into a numerical format that machine learning algorithms can work with effectively 
# and joininig them with df2
df2 = df2.join(pd.get_dummies(df2[['Role', 'Team']]))

In [40]:
# define decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status', ((obj) for obj in df2.index), cat = 'Binary')

# define objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Points'] for obj in df2.index])

# define constraints
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Credits']] for obj in df2.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'PlayingXI']] for obj in df2.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Team_IND']] for obj in df2.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Team_IND']] for obj in df2.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Team_SL']] for obj in df2.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Team_SL']] for obj in df2.index) <= 7

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_WK']] for obj in df2.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_WK']] for obj in df2.index) <= 3

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BAT']] for obj in df2.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BAT']] for obj in df2.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BOWL']] for obj in df2.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BOWL']] for obj in df2.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_AR']] for obj in df2.index) >= 2
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_AR']] for obj in df2.index) <= 4



In [42]:
# solving the linear programming problem for the given constraints
mo.solve()

1

In [44]:
# Sorting df2 in descending order of Points
df2.sort_values('Points', ascending = False)

,Player,Team,Role,Credits,Points,PlayingXI,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,S Iyer,IND,BAT,8.5,112,1,False,True,False,False,True,False
1,P Nissanka,SL,BAT,9.5,100,1,False,True,False,False,False,True
2,R Jadeja,IND,AR,9.0,93,1,True,False,False,False,True,False
3,D Shanaka,SL,AR,9.0,77,1,True,False,False,False,False,True
4,S Samson,IND,WK,8.0,59,1,False,False,False,True,True,False
5,D Gunathilaka,SL,BAT,8.5,56,1,False,True,False,False,False,True
6,L Kumara,SL,BOWL,8.5,52,1,False,False,True,False,False,True
7,Y Chahal,IND,BOWL,8.5,39,1,False,False,True,False,True,False
8,B Kumar,IND,BOWL,8.5,37,1,False,False,True,False,True,False
9,D Chameera,SL,BOWL,9.0,31,1,False,False,True,False,False,True


In [46]:
# output as a column in df
df2['pickup_status'] = 0

for obj in df2.index:
    if pickup_status[obj].varValue:
        df2['pickup_status'][obj] = 1

# Keeping only the players picked up in the fantasy team

fdf2 = df2[df2['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)

#### 2nd T20I Fantasy Team

In [49]:
fdf2

,Player,Team,Role,Credits,Points,PlayingXI,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,S Iyer,IND,BAT,8.5,112,1,False,True,False,False,True,False,1
1,P Nissanka,SL,BAT,9.5,100,1,False,True,False,False,False,True,1
2,D Gunathilaka,SL,BAT,8.5,56,1,False,True,False,False,False,True,1
3,S Samson,IND,WK,8.0,59,1,False,False,False,True,True,False,1
4,R Jadeja,IND,AR,9.0,93,1,True,False,False,False,True,False,1
5,D Shanaka,SL,AR,9.0,77,1,True,False,False,False,False,True,1
6,L Kumara,SL,BOWL,8.5,52,1,False,False,True,False,False,True,1
7,Y Chahal,IND,BOWL,8.5,39,1,False,False,True,False,True,False,1
8,B Kumar,IND,BOWL,8.5,37,1,False,False,True,False,True,False,1
9,D Chameera,SL,BOWL,9.0,31,1,False,False,True,False,False,True,1


In [51]:
sum(fdf2.Credits)

96.0

In [53]:
# Total Points in the second fanatsy team
F2TPS = sum(fdf2['Points'])

F2TPS

687

#### IND VS SL 3RD T20I

In [56]:
# Filtering df3 to select the players announced in Playing XI
df3 = df3[df3['PlayingXI'] == 1]

In [58]:
# converting team names and Roles into a numerical format that machine learning algorithms can work with effectively 
# and joininig them with df3
df3 = df3.join(pd.get_dummies(df3[['Role', 'Team']]))

In [60]:
df3

,Player,Team,Role,Credits,Points,PlayingXI,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL
0,S Iyer,IND,BAT,8.5,100,1,False,True,False,False,True,False
1,R Jadeja,IND,AR,9.0,31,1,True,False,False,False,True,False
2,S Samson,IND,WK,8.0,35,1,False,False,False,True,True,False
3,D Shanaka,SL,AR,9.0,105,1,True,False,False,False,False,True
4,P Nissanka,SL,BAT,9.5,-1,1,False,True,False,False,False,True
5,R Sharma,IND,BAT,10.5,10,1,False,True,False,False,True,False
6,D Chameera,SL,BOWL,9.0,31,1,False,False,True,False,False,True
7,D Gunathilaka,SL,BAT,8.5,2,1,False,True,False,False,False,True
8,L Kumara,SL,BOWL,8.5,60,1,False,False,True,False,False,True
9,C Asalanka,SL,BAT,8.5,9,1,False,True,False,False,False,True


In [62]:
# define decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status', ((obj) for obj in df3.index), cat = 'Binary')

# define objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Points'] for obj in df3.index])

# define constraints
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Credits']] for obj in df3.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'PlayingXI']] for obj in df3.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Team_IND']] for obj in df3.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Team_IND']] for obj in df3.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Team_SL']] for obj in df3.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Team_SL']] for obj in df3.index) <= 7

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_WK']] for obj in df3.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_WK']] for obj in df3.index) <= 3

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_BAT']] for obj in df3.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_BAT']] for obj in df3.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_BOWL']] for obj in df3.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_BOWL']] for obj in df3.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_AR']] for obj in df3.index) >= 2
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_AR']] for obj in df3.index) <= 4



In [64]:
# solving the linear programming problem for the given constraints
mo.solve()

1

In [66]:
# output as a column in df
df3['pickup_status'] = 0

for obj in df3.index:
    if pickup_status[obj].varValue:
        df3['pickup_status'][obj] = 1

# Keeping only the players picked up in the fantasy team

fdf3 = df3[df3['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)

#### 3rd T20I Fantasy Team

In [69]:
fdf3

,Player,Team,Role,Credits,Points,PlayingXI,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,S Iyer,IND,BAT,8.5,100,1,False,True,False,False,True,False,1
1,R Sharma,IND,BAT,10.5,10,1,False,True,False,False,True,False,1
2,C Asalanka,SL,BAT,8.5,9,1,False,True,False,False,False,True,1
3,S Samson,IND,WK,8.0,35,1,False,False,False,True,True,False,1
4,D Chandimal,SL,WK,8.5,36,1,False,False,False,True,False,True,1
5,D Shanaka,SL,AR,9.0,105,1,True,False,False,False,False,True,1
6,C Karunaratne,SL,AR,8.5,47,1,True,False,False,False,False,True,1
7,L Kumara,SL,BOWL,8.5,60,1,False,False,True,False,False,True,1
8,R Bishnoi,IND,BOWL,8.0,37,1,False,False,True,False,True,False,1
9,M Siraj,IND,BOWL,8.5,41,1,False,False,True,False,True,False,1


In [71]:
sum(fdf3.Credits)

94.5

In [73]:
F3TPS = sum(fdf3['Points'])

F3TPS

550

In [75]:
#### Total Fantasy Points of 3 T20Is

In [77]:
F1TPS, F2TPS, F3TPS

(653, 687, 550)

In [79]:
FTPS = F1TPS + F2TPS + F3TPS

In [81]:
# Sum total of Fantasy Points of all three T20Is
FTPS


1890

In [83]:
# Average Fantasy Points per match
FTPS/3

630.0

#### Average Fantasy Points per match is 630